In [1]:
import tweepy #https://github.com/tweepy/tweepy
import csv
import pandas as pd
import numpy as np

In [2]:
#Twitter API credentials
consumer_key = ""
consumer_secret = ""
access_key = ""
access_secret = ""

In [137]:
pd.set_option('display.max_colwidth', -1)
def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method
    
    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    #initialize a list to hold all the tweepy Tweets
    alltweets = []
    
    tweet_dict = {}
    columns = ['ID', 'Created At', 'Tweet', 'tree', 'grass', 'mold', 'weed', 'ragweed']
    for col in columns:
        tweet_dict[col] = []
    
    pollen_full_list = ['tree', 'grass', 'mold', 'weed', 'ragweed']
    pollen_day = []
    level_list = ['absent', 'low', 'moderate', 'high']

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200, tweet_mode = "extended")
    
    #save most recent tweets
    alltweets.extend(new_tweets)
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name, count=200, max_id=oldest, tweet_mode = "extended")

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print("..." +str(len(alltweets))+ " tweets downloaded so far." )
    
    #transform the tweepy tweets into a 2D array that will populate the csv	
    # outtweets = [[tweet.id_str, tweet.created_at.strftime('%Y-%m-%d'), str(tweet.full_text)] for tweet in alltweets]
    for tweet in alltweets:
        tweet_dict['ID'].append(tweet.id_str)
        tweet_dict['Created At'].append(tweet.created_at.strftime('%Y-%m-%d'))
        tweet_dict['Tweet'].append(str(tweet.full_text))
        # tweet.extend((tweet.full_text))
        pollen_list = get_pollen_levels_list(tweet.full_text)
        pollen_day = []
        print(tweet.full_text)
        
        for pollen in pollen_list:
            pollen_lst = []
            
            pollen_lst.extend(re.findall(r"tree|grass|mold|weed|ragweed", pollen.lower()))
            # print(pollen_lst)
            pollen_lst = list(set(pollen_lst))
            # print(pollen)
            level = re.findall(r"absent|low|moderate|high|not present", pollen.lower())
            for p in pollen_lst:
                if p not in pollen_day:
                    if len(level)>0:
                        print(p, level[0])
                        if level[0] != '':
                            tweet_dict[p].append(level[0])
                            pollen_day.append(p)
                
                    # print(df_id, p, level[0])            
            # print(pollen_day)
        for p in pollen_full_list:
            if p not in pollen_day:
                tweet_dict[p].append('absent')
                
        print(pollen_day)
        for key, val in tweet_dict.items():
            print(key, len(val))
        print('-'*40)
    # tweets_df = pd.DataFrame(outtweets, columns= ['ID', 'Created At', 'Tweet'])
    #write the csv
    # tweets_df.set_index('ID')

    '''
    with open('%s_tweets.csv' % screen_name, 'wb') as f:
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text"])
        writer.writerows(outtweets)

    pass
    '''
    # print(tweets_df['Tweet'])
    return tweet_dict
    # print(datetime(tweet.created_at) for tweet in outtweets)

In [151]:
if __name__ == '__main__':
    #pass in the username of the account you want to download
    tweets = get_all_tweets("GottliebAllergy")
    print(tweets)

...400 tweets downloaded so far.
...600 tweets downloaded so far.
...800 tweets downloaded so far.
...1000 tweets downloaded so far.
...1200 tweets downloaded so far.
...1400 tweets downloaded so far.
...1506 tweets downloaded so far.
...1506 tweets downloaded so far.
9-30-19
This is the last pollen count for the season. We are happy to return to provide this service next spring.

Good morning 
Pollen count for today
Mold moderate
Total weeds low
Ragweed low
mold moderate
weed low
ragweed low
['mold', 'weed', 'ragweed']
ID 1
Created At 1
Tweet 1
tree 1
grass 1
mold 1
weed 1
ragweed 1
----------------------------------------
Pollen count for today 9-27-2019

Grass low
Mold high
Total weeds Moderate
Ragweed Moderate
grass low
mold high
weed moderate
ragweed moderate
['grass', 'mold', 'weed', 'ragweed']
ID 2
Created At 2
Tweet 2
tree 2
grass 2
mold 2
weed 2
ragweed 2
----------------------------------------
9-26-2019

Trees- absent
Grass - absent
Molds - high
Weeds - low
Ragweed - low

Th

In [152]:
for key, val in tweets.items():
    print(key, len(val))

ID 1506
Created At 1506
Tweet 1506
tree 1506
grass 1506
mold 1506
weed 1506
ragweed 1506


In [153]:
tweets_df = pd.DataFrame(tweets)
tweets_df

,ID,Created At,Tweet,tree,grass,mold,weed,ragweed
0,1178661333933408256,2019-09-30,9-30-19\nThis is the last pollen count for the season. We are happy to return to provide this service next spring.\n\nGood morning \nPollen count for today\nMold moderate\nTotal weeds low\nRagweed low,absent,absent,moderate,low,low
1,1177594784854216705,2019-09-27,Pollen count for today 9-27-2019\n\nGrass low\nMold high\nTotal weeds Moderate\nRagweed Moderate,absent,low,high,moderate,moderate
2,1177209411124760576,2019-09-26,9-26-2019\n\nTrees- absent\nGrass - absent\nMolds - high\nWeeds - low\nRagweed - low\n\nThank you. The last report will be Monday 9/30,absent,absent,high,low,low
3,1176863255261786112,2019-09-25,Allergy report for 9-25-19\n\nTrees - absent\nGrass - low\nMolds - high\nWeeds - low\nRagweed - low\n\nReminder: Final count Monday September 30th,absent,low,high,low,low
4,1176492992057794561,2019-09-24,Allergy report for 9-24-2019. \n\nTrees - absent\nGrass - low\nMolds - high\nWeeds- moderate\nRagweed - low\n\nNote : Monday 9/30/19 will be our final reporting for this season. We'll return in the spring.,absent,low,high,moderate,low
5,1176122305178722304,2019-09-23,"Allergy report for Sept 23, 2019\n\nTrees- absent\nGrass - absent\nMolds - high\nWeeds - moderate\nRagweed - low",absent,absent,high,moderate,low
6,1175043252128550913,2019-09-20,Allergy Count for 9--20-19\n\nGrass low\nWeeds high\nRagweed high\nMold high,absent,low,high,high,high
7,1174311640847794177,2019-09-18,9-18-2019\nToday's allergy count\n\nTrees - absent\nGrass - low\nMolds - high\nWeeds - moderate\nRagweed - moderate,absent,low,high,moderate,moderate
8,1173951816620486656,2019-09-17,Loyola Medicine Allergy Count\n\nTrees - absent\nGrass - low\nMolds - moderate\nWeeds - moderate\nRagweed - low\n\nThank you,absent,low,moderate,moderate,low
9,1173569052150763520,2019-09-16,Loyola Medicine Allergy Count \n9-16-19\n\nTrees - absent Grass- low\nMold - high\nWeeds - high\nRagweed - moderate,absent,absent,high,high,moderate


In [154]:
tweets_df[['Created At', 'tree', 'grass', 'mold', 'weed', 'ragweed']].to_csv('PollenLevels_.csv', sep='|', index = False )

In [150]:
import re

def get_pollen_levels_list(string):
    # string = string.replace('')
    split_str = re.split('\n|;', string)
    # print(split_str)
    pollen_list = ['tree', 'grass', 'mold', 'weed', 'ragweed']
    str_pollen = [s for s in split_str if any(xs in s.lower() for xs in pollen_list)]
    
    return str_pollen

In [9]:
tweet_1 = tweets.loc[tweets['ID'] == '1177594784854216705']
print(tweet_1)

                    ID  Created At  \
1  1177594784854216705  2019-09-27   

                                                                                              Tweet  \
1  Pollen count for today 9-27-2019\n\nGrass low\nMold high\nTotal weeds Moderate\nRagweed Moderate   

   tree  grass  mold  weed  ragweed  
1 NaN   NaN    NaN   NaN   NaN       


In [10]:
tweet_1['Levels'] = tweet_1['Tweet'].apply(get_pollen_levels_list)
print(tweet_1)

                    ID  Created At  \
1  1177594784854216705  2019-09-27   

                                                                                              Tweet  \
1  Pollen count for today 9-27-2019\n\nGrass low\nMold high\nTotal weeds Moderate\nRagweed Moderate   

   tree  grass  mold  weed  ragweed  \
1 NaN   NaN    NaN   NaN   NaN        

                                                           Levels  
1  [Grass low, Mold high, Total weeds Moderate, Ragweed Moderate]  


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
import re
def get_individual_pollen_levels(df):
    pollen_list = ['tree', 'grass', 'mold', 'weed', 'ragweed']
    level_list = ['absent', 'low', 'moderate', 'high']
    # print(pollen_tweet_list)
    pollen_level_list = []
    print(df['ID'], df['Levels'])
    df_id = str(df['ID'])
    # 
    for tweet_level in list(df['Levels']):

        pollen = []
        '''
        for pollen in pollen_list:
            regex_str = pollen+' '
            print(regex_str, tweet_level.lower())
            if re.match(regex_str, tweet_level.lower()):
                # print(tweet_level)
                for level in level_list:
                    if level in tweet_level.lower():
                        # print(pollen, level)
                        tweet_1[pollen] = level
         '''
        
        pollen.extend(re.findall(r"tree|grass|mold|weed|ragweed", tweet_level.lower()))
        # print(pollen)
        level = re.findall(r"absent|low|moderate|high", tweet_level.lower())
        # print(pollen, level)
        for p in pollen:
            if len(level)>0:
                print(df_id, p, level[0])
                df.loc[int(df_id), p] = level[0]
                
# tweet_1.apply(get_individual_pollen_levels, axis = 1)

In [12]:
tweet_1[['ID', 'Levels']]

,ID,Levels
1,1177594784854216705,"[Grass low, Mold high, Total weeds Moderate, Ragweed Moderate]"


In [13]:
tweets.head(20)

,ID,Created At,Tweet,tree,grass,mold,weed,ragweed
0,1178661333933408256,2019-09-30,9-30-19\nThis is the last pollen count for the season. We are happy to return to provide this service next spring.\n\nGood morning \nPollen count for today\nMold moderate\nTotal weeds low\nRagweed low,NaN,NaN,NaN,NaN,NaN
1,1177594784854216705,2019-09-27,Pollen count for today 9-27-2019\n\nGrass low\nMold high\nTotal weeds Moderate\nRagweed Moderate,NaN,NaN,NaN,NaN,NaN
2,1177209411124760576,2019-09-26,9-26-2019\n\nTrees- absent\nGrass - absent\nMolds - high\nWeeds - low\nRagweed - low\n\nThank you. The last report will be Monday 9/30,NaN,NaN,NaN,NaN,NaN
3,1176863255261786112,2019-09-25,Allergy report for 9-25-19\n\nTrees - absent\nGrass - low\nMolds - high\nWeeds - low\nRagweed - low\n\nReminder: Final count Monday September 30th,NaN,NaN,NaN,NaN,NaN
4,1176492992057794561,2019-09-24,Allergy report for 9-24-2019. \n\nTrees - absent\nGrass - low\nMolds - high\nWeeds- moderate\nRagweed - low\n\nNote : Monday 9/30/19 will be our final reporting for this season. We'll return in the spring.,NaN,NaN,NaN,NaN,NaN
5,1176122305178722304,2019-09-23,"Allergy report for Sept 23, 2019\n\nTrees- absent\nGrass - absent\nMolds - high\nWeeds - moderate\nRagweed - low",NaN,NaN,NaN,NaN,NaN
6,1175043252128550913,2019-09-20,Allergy Count for 9--20-19\n\nGrass low\nWeeds high\nRagweed high\nMold high,NaN,NaN,NaN,NaN,NaN
7,1174311640847794177,2019-09-18,9-18-2019\nToday's allergy count\n\nTrees - absent\nGrass - low\nMolds - high\nWeeds - moderate\nRagweed - moderate,NaN,NaN,NaN,NaN,NaN
8,1173951816620486656,2019-09-17,Loyola Medicine Allergy Count\n\nTrees - absent\nGrass - low\nMolds - moderate\nWeeds - moderate\nRagweed - low\n\nThank you,NaN,NaN,NaN,NaN,NaN
9,1173569052150763520,2019-09-16,Loyola Medicine Allergy Count \n9-16-19\n\nTrees - absent Grass- low\nMold - high\nWeeds - high\nRagweed - moderate,NaN,NaN,NaN,NaN,NaN


In [14]:
tweets['Levels'] = tweets['Tweet'].apply(get_pollen_levels_list)
tweets.head(20)

,ID,Created At,Tweet,tree,grass,mold,weed,ragweed,Levels
0,1178661333933408256,2019-09-30,9-30-19\nThis is the last pollen count for the season. We are happy to return to provide this service next spring.\n\nGood morning \nPollen count for today\nMold moderate\nTotal weeds low\nRagweed low,NaN,NaN,NaN,NaN,NaN,"[Mold moderate, Total weeds low, Ragweed low]"
1,1177594784854216705,2019-09-27,Pollen count for today 9-27-2019\n\nGrass low\nMold high\nTotal weeds Moderate\nRagweed Moderate,NaN,NaN,NaN,NaN,NaN,"[Grass low, Mold high, Total weeds Moderate, Ragweed Moderate]"
2,1177209411124760576,2019-09-26,9-26-2019\n\nTrees- absent\nGrass - absent\nMolds - high\nWeeds - low\nRagweed - low\n\nThank you. The last report will be Monday 9/30,NaN,NaN,NaN,NaN,NaN,"[Trees- absent, Grass - absent, Molds - high, Weeds - low, Ragweed - low]"
3,1176863255261786112,2019-09-25,Allergy report for 9-25-19\n\nTrees - absent\nGrass - low\nMolds - high\nWeeds - low\nRagweed - low\n\nReminder: Final count Monday September 30th,NaN,NaN,NaN,NaN,NaN,"[Trees - absent, Grass - low, Molds - high, Weeds - low, Ragweed - low]"
4,1176492992057794561,2019-09-24,Allergy report for 9-24-2019. \n\nTrees - absent\nGrass - low\nMolds - high\nWeeds- moderate\nRagweed - low\n\nNote : Monday 9/30/19 will be our final reporting for this season. We'll return in the spring.,NaN,NaN,NaN,NaN,NaN,"[Trees - absent, Grass - low, Molds - high, Weeds- moderate, Ragweed - low]"
5,1176122305178722304,2019-09-23,"Allergy report for Sept 23, 2019\n\nTrees- absent\nGrass - absent\nMolds - high\nWeeds - moderate\nRagweed - low",NaN,NaN,NaN,NaN,NaN,"[Trees- absent, Grass - absent, Molds - high, Weeds - moderate, Ragweed - low]"
6,1175043252128550913,2019-09-20,Allergy Count for 9--20-19\n\nGrass low\nWeeds high\nRagweed high\nMold high,NaN,NaN,NaN,NaN,NaN,"[Grass low, Weeds high, Ragweed high, Mold high]"
7,1174311640847794177,2019-09-18,9-18-2019\nToday's allergy count\n\nTrees - absent\nGrass - low\nMolds - high\nWeeds - moderate\nRagweed - moderate,NaN,NaN,NaN,NaN,NaN,"[Trees - absent, Grass - low, Molds - high, Weeds - moderate, Ragweed - moderate]"
8,1173951816620486656,2019-09-17,Loyola Medicine Allergy Count\n\nTrees - absent\nGrass - low\nMolds - moderate\nWeeds - moderate\nRagweed - low\n\nThank you,NaN,NaN,NaN,NaN,NaN,"[Trees - absent, Grass - low, Molds - moderate, Weeds - moderate, Ragweed - low]"
9,1173569052150763520,2019-09-16,Loyola Medicine Allergy Count \n9-16-19\n\nTrees - absent Grass- low\nMold - high\nWeeds - high\nRagweed - moderate,NaN,NaN,NaN,NaN,NaN,"[Trees - absent Grass- low, Mold - high, Weeds - high, Ragweed - moderate]"


In [15]:
tweets['Levels'].apply(get_individual_pollen_levels)
tweets.head(20)

TypeError: list indices must be integers or slices, not str

In [22]:
dict_test = {}
dict_test['Test'] = []
dict_test['Test'].extend('A')
dict_test['Test'].extend('B')
dict_test

{'Test': ['A', 'B']}

In [24]:
string = 'Trees - Moderate; Mold - Low; Weeds - Moderate'
pollen = re.findall(r"tree|grass|mold|weed|ragweed", string.lower())
# print(pollen)
level = re.findall(r"absent|low|moderate|high", string.lower())
print(pollen, level)
for i in range(0)

['tree', 'mold', 'weed'] ['moderate', 'low', 'moderate']


In [27]:
l1= ['a', 'b', 'c', 'd']
l2= ['a', 'c']
l3= []
for l in l1:
    if l not in l2:
        l3.extend(l)
print(l3)

['b', 'd']


In [149]:
import re

string = '9-30-19\nThis is the last pollen count for the season. We are happy to return to provide this service next spring.\n\nGood morning \nPollen count for today\nMold moderate\nTotal weeds low\nRagweed low'
print(string)
print('-'*40)
lst = re.split('\n|;', string)
for l in lst:
    print(l)
    print('-'*10)

9-30-19
This is the last pollen count for the season. We are happy to return to provide this service next spring.

Good morning 
Pollen count for today
Mold moderate
Total weeds low
Ragweed low
----------------------------------------
9-30-19
----------
This is the last pollen count for the season. We are happy to return to provide this service next spring.
----------

----------
Good morning 
----------
Pollen count for today
----------
Mold moderate
----------
Total weeds low
----------
Ragweed low
----------
